In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_curve, auc
import seaborn as sns
from sklearn.tree import export_graphviz
from IPython.display import Image
from graphviz import Source
from IPython.display import Image
import graphviz
from sklearn.model_selection import cross_val_score
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV


In [2]:
path = "C:/Users/Wojtek/Desktop/NBA_jazda.xlsx"

df = pd.read_excel(path)

df = df.drop(df.columns[0], axis=1)
df


,Team_Home,Team_Away,Date,Season,Home_score,Home_FG_PCT,Home_FG3_PCT,Home_FT_PCT,Home_REB,Home_AST,...,Away_REB,Away_AST,Away_TOV,Away_STL,Away_BLK,Away_OFF_RATING,Away_DEF_RATING,Away_TS_PCT,RESULT_Home,RESULT_Away
0,Boston Celtics,Philadelphia 76ers,2018-10-16,2018-19,105,0.433,0.297,0.714,55,21,...,47,18,16,8,5,83.4,100.7,0.448,1,0
1,Golden State Warriors,Oklahoma City Thunder,2018-10-16,2018-19,108,0.442,0.269,0.944,58,28,...,45,21,14,12,6,97.7,105.6,0.466,1,0
2,Charlotte Hornets,Milwaukee Bucks,2018-10-17,2018-19,112,0.446,0.421,0.636,41,21,...,57,26,21,5,4,110.2,109.2,0.602,0,1
3,Indiana Pacers,Memphis Grizzlies,2018-10-17,2018-19,111,0.566,0.385,0.538,57,29,...,28,16,7,11,3,88.8,118.7,0.431,1,0
4,Los Angeles Clippers,Denver Nuggets,2018-10-17,2018-19,98,0.398,0.286,0.833,47,21,...,56,20,10,6,9,108.2,99.1,0.507,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,Orlando Magic,New Orleans Pelicans,2020-08-13,2019-20,133,0.571,0.536,0.733,33,25,...,40,22,13,6,4,125.7,131.6,0.642,1,0
2281,Utah Jazz,San Antonio Spurs,2020-08-13,2019-20,118,0.465,0.326,0.900,43,25,...,44,24,15,4,8,113.7,119.8,0.590,1,0
2282,Memphis Grizzlies,Milwaukee Bucks,2020-08-13,2019-20,119,0.500,0.394,0.696,55,36,...,45,27,15,8,3,96.6,108.5,0.490,1,0
2283,Boston Celtics,Washington Wizards,2020-08-13,2019-20,90,0.398,0.229,0.857,51,22,...,56,22,18,11,6,98.9,92.7,0.487,0,1


In [3]:
columns_to_scale = ['Home_REB', 'Home_AST','Home_OFF_RATING',
       'Home_DEF_RATING','Home_TOV', 'Home_STL', 'Home_BLK', 'Home_OFF_RATING',
       'Home_DEF_RATING', 'Away_REB', 'Away_AST', 'Away_TOV', 'Away_STL',
       'Away_BLK','Away_OFF_RATING', 'Away_DEF_RATING']

In [4]:
scaler = MinMaxScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [5]:
df['Shifted_1'] = df['RESULT_Home'].shift(1)
df['Shifted_2'] = df['Home_FG_PCT'].shift(1)
df['Shifted_3'] = df['Home_FG3_PCT'].shift(1)
df['Shifted_4'] = df['Home_FT_PCT'].shift(1)
df['Shifted_5'] = df['Home_REB'].shift(1)
df['Shifted_6'] = df['Home_AST'].shift(1)
df['Shifted_7'] = df['Home_TOV'].shift(1)
df['Shifted_8'] = df['Home_STL'].shift(1)
df['Shifted_9'] = df['Home_BLK'].shift(1)
df['Shifted_10'] = df['Home_OFF_RATING'].shift(1)
df['Shifted_11'] = df['Home_DEF_RATING'].shift(1)
df['Shifted_12'] = df['Home_TS_PCT'].shift(1)
df['Shifted_13'] = df['RESULT_Away'].shift(1)
df['Shifted_14'] = df['Away_FG_PCT'].shift(1)
df['Shifted_15'] = df['Away_FG3_PCT'].shift(1)
df['Shifted_16'] = df['Away_FT_PCT'].shift(1)
df['Shifted_17'] = df['Away_REB'].shift(1)
df['Shifted_18'] = df['Away_AST'].shift(1)
df['Shifted_19'] = df['Away_TOV'].shift(1)
df['Shifted_20'] = df['Away_STL'].shift(1)
df['Shifted_21'] = df['Away_BLK'].shift(1)
df['Shifted_22'] = df['Away_OFF_RATING'].shift(1)
df['Shifted_23'] = df['Away_DEF_RATING'].shift(1)
df['Shifted_24'] = df['Away_TS_PCT'].shift(1)


In [6]:
selected_columns = ['Home_FG_PCT', 'Home_FG3_PCT', 'Home_FT_PCT', 'Home_REB', 'Home_AST',
       'Home_TOV', 'Home_STL', 'Home_BLK', 
       'Home_OFF_RATING', 'Home_DEF_RATING', 'Home_TS_PCT', 'RESULT_Home']

In [7]:
df_rolling = df[list(selected_columns) + ["RESULT_Home", "Team_Home", "Season"]]

In [8]:
def find_Team_Home_avarages(Team_Home):
    rolling = Team_Home.rolling(10).mean()  #jeżeli zmienimy liczbę przy team rolling to wtedy będzie brało średnie z innej liczby gier
    return rolling

df_rolling = df_rolling.groupby(["Team_Home", "Season"], group_keys=False).apply(find_Team_Home_avarages)


C:\Users\Wojtek\AppData\Local\Temp\ipykernel_3176\66027910.py:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Season', 'Team_Home'], dtype='object')
  rolling = Team_Home.rolling(10).mean()  #jeżeli zmienimy liczbę przy team rolling to wtedy będzie brało średnie z innej liczby gier


In [9]:
rolling_cols = [f"{col}_10" for col in df_rolling.columns]  #doda nam 10 do każdej kolumny oznaczającej średnia z 10 gier
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis=1)   #axis = 1 oznacza że używamy kolumn do połączenia

In [10]:
selected_columns_1 = ['Away_FG_PCT',
       'Away_FG3_PCT', 'Away_FT_PCT', 'Away_REB', 'Away_AST', 'Away_TOV',
       'Away_STL', 'Away_BLK', 'Away_OFF_RATING',
       'Away_DEF_RATING', 'Away_TS_PCT']

In [11]:
df_rolling_1 = df[list(selected_columns) + ["RESULT_Away", "Team_Away", "Season"]]

In [12]:
def find_Team_Away_avarages(Team_Away):
    rolling_1 = Team_Away.rolling(10).mean()  #jeżeli zmienimy liczbę przy team rolling to wtedy będzie brało średnie z innej liczby gier
    return rolling_1

df_rolling_1 = df_rolling_1.groupby(["Team_Away", "Season"], group_keys=False).apply(find_Team_Away_avarages)

C:\Users\Wojtek\AppData\Local\Temp\ipykernel_3176\2438606815.py:2: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['Season', 'Team_Away'], dtype='object')
  rolling_1 = Team_Away.rolling(10).mean()  #jeżeli zmienimy liczbę przy team rolling to wtedy będzie brało średnie z innej liczby gier


In [13]:
rolling_cols_1 = [f"{col}_10" for col in df_rolling_1.columns]  #doda nam 10 do każdej kolumny oznaczającej średnia z 10 gier
df_rolling_1.columns = rolling_cols_1

df = pd.concat([df, df_rolling_1], axis=1)   #axis = 1 oznacza że używamy kolumn do połączenia

In [14]:
df = df.drop(columns= ['Shifted_12','Shifted_11', 'Shifted_10','Shifted_9','Shifted_8','Shifted_7',
                       'Shifted_6','Shifted_5','Shifted_4','Shifted_3','Shifted_2', 'Shifted_1','Shifted_13','Shifted_14', 'Shifted_15','Shifted_16','Shifted_17','Shifted_18',
                       'Shifted_19','Shifted_20','Shifted_21','Shifted_22','Shifted_23', 'Shifted_24'])

In [15]:
final_df = df.drop(columns = ["Team_Home", "Team_Away",  "Date", "Season" , "Home_score", "Away_score"], axis = 1, inplace=True)

In [16]:
df_model = df.dropna()

In [17]:
df.drop(columns=['Home_FG_PCT', 'Home_FG3_PCT', 'Home_FT_PCT', 'Home_REB', 'Home_AST',
       'Home_TOV', 'Home_STL', 'Home_BLK', 
       'Home_OFF_RATING', 'Home_DEF_RATING', 'Home_TS_PCT', 'Away_FG_PCT',
       'Away_FG3_PCT', 'Away_FT_PCT', 'Away_REB', 'Away_AST', 'Away_TOV',
       'Away_STL', 'Away_BLK', 'Away_OFF_RATING',
       'Away_DEF_RATING', 'Away_TS_PCT', 'RESULT_Away'], axis = 1, inplace=True)

In [18]:
df_model = df.dropna()

In [19]:
df_dra = df_model.dropna()

In [20]:
df_dra

,RESULT_Home,Home_FG_PCT_10,Home_FG3_PCT_10,Home_FT_PCT_10,Home_REB_10,Home_AST_10,Home_TOV_10,Home_STL_10,Home_BLK_10,Home_OFF_RATING_10,...,Home_REB_10,Home_AST_10,Home_TOV_10,Home_STL_10,Home_BLK_10,Home_OFF_RATING_10,Home_DEF_RATING_10,Home_TS_PCT_10,RESULT_Home_10,RESULT_Away_10
237,1,0.4752,0.3340,0.7843,0.387500,0.475758,0.452,0.380,0.315,0.422049,...,0.531250,0.484848,0.512,0.330,0.315,0.527108,0.506210,0.5976,0.8,0.2
248,0,0.4331,0.3730,0.7032,0.550000,0.351515,0.496,0.300,0.360,0.391699,...,0.545833,0.327273,0.444,0.335,0.260,0.426070,0.520659,0.5406,0.5,0.5
272,1,0.5075,0.4268,0.8033,0.475000,0.636364,0.472,0.375,0.325,0.533074,...,0.495833,0.463636,0.380,0.400,0.325,0.475616,0.518124,0.5642,0.5,0.5
278,1,0.4930,0.3367,0.7847,0.612500,0.518182,0.476,0.330,0.295,0.519974,...,0.487500,0.518182,0.400,0.385,0.215,0.496109,0.553105,0.5812,0.7,0.3
279,1,0.4895,0.3803,0.7395,0.456250,0.442424,0.492,0.405,0.210,0.516732,...,0.510417,0.381818,0.496,0.340,0.200,0.380804,0.492522,0.5460,0.6,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2280,1,0.4565,0.3474,0.8006,0.477083,0.533333,0.356,0.425,0.210,0.466018,...,0.454167,0.457576,0.440,0.415,0.330,0.480545,0.555260,0.5845,0.6,0.4
2281,1,0.4545,0.3623,0.8117,0.418750,0.378788,0.448,0.310,0.200,0.472633,...,0.497917,0.493939,0.424,0.255,0.210,0.502335,0.562864,0.5867,0.5,0.5
2282,1,0.4726,0.2935,0.7644,0.583333,0.518182,0.420,0.350,0.275,0.431777,...,0.525000,0.500000,0.364,0.350,0.220,0.405966,0.458048,0.5479,0.7,0.3
2283,0,0.4421,0.3738,0.7990,0.518750,0.421212,0.436,0.430,0.335,0.447471,...,0.522917,0.412121,0.564,0.430,0.270,0.494812,0.532319,0.5931,0.7,0.3


In [ ]:
X = df_dra.drop(columns = 'RESULT_Home')

y = df_dra['RESULT_Home']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X , y , test_size = 0.2, random_state = 42)

In [ ]:
accuracy_scores = []
num_folds = 10
for i in range(num_folds):
    X_train, X_test, y_train, y_test = train_test_split( X , y , test_size = 0.2, random_state = 42)

    rf_model = RandomForestClassifier(max_depth=8, max_leaf_nodes=962, min_samples_split=92, n_estimators=1519, random_state=42)

    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    accuracy_scores.append(accuracy)
  
mean_accuracy = np.mean(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)                

In [ ]:
y_train_pred = rf_model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)


print("Training Accuracy:", train_accuracy)


In [ ]:
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

auc_score = auc(fpr, tpr)

plt.figure(figsize=(7, 7), dpi=500)
plt.plot(fpr, tpr, marker='.', label='Random Forest (auc = %0.3f)' % auc_score)

plt.xlabel('False Positive Rate -->')
plt.ylabel('True Positive Rate -->')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Random Guess')
plt.legend()

plt.show()


In [ ]:
y_pred = rf_model.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, cmap='Greens', fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('True Label')
plt.ylabel('Predicted Label')
plt.show()

In [ ]:
sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])

specificity = cm[1,1]/(cm[1,1]+cm[0,1])

print('Sensitivity:', sensitivity)
print('Specificity:', specificity)


In [ ]:
scores = cross_val_score(rf_model, X, y, cv=10)
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())

In [ ]:
for i in range(3):
    tree = rf_model.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=X_train.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
param_dist = {'n_estimators': randint(1,2000),
              'max_depth': randint(1,20),
             'min_samples_split' :  randint(1,1000),
             'max_leaf_nodes' : randint(1,1000) }


rf_model = RandomForestClassifier()

rand_search = RandomizedSearchCV(rf_model, 
                                 param_distributions = param_dist, 
                                 n_iter=5, 
                                 cv=5)

rand_search.fit(X_train, y_train)

In [ ]:
best_rf = rand_search.best_estimator_

print('Best hyperparameters:',  rand_search.best_params_)

In [ ]:
from sklearn.model_selection import cross_val_score
rf_model = RandomForestClassifier(max_depth=7, max_leaf_nodes=216, min_samples_split=140, n_estimators=299, random_state=42)
scores = cross_val_score(rf_model, X, y, cv=10)
print("Mean cross-validation score:", scores.mean())
print("Standard deviation of cross-validation scores:", scores.std())